<a href="https://colab.research.google.com/github/TAFIMnoor/Machine-Learning-Lab/blob/main/Final%20Lab%20Assignments/SVM_Bank_Deposit_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip drive/My\ Drive/bank-data.zip

Archive:  drive/My Drive/bank-data.zip
   creating: bank-data/
  inflating: __MACOSX/._bank-data    
  inflating: bank-data/.DS_Store     
  inflating: __MACOSX/bank-data/._.DS_Store  
  inflating: bank-data/bank-full.csv  
  inflating: __MACOSX/bank-data/._bank-full.csv  


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [13]:
data_dir = '/content/bank-data/bank-full.csv'

df = pd.read_csv(data_dir, delimiter=';')

# Preprocessing the data
df.dropna(inplace=True)  # remove rows with missing values
df = df.drop(columns=['contact', 'day','month']) #remove columns


In [28]:
df

,age,job,marital,education,default,balance,housing,loan,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,508,4,-1,0,unknown,no


In [29]:
X = df.drop('y', axis=1)  # extract features
y = df['y']  # extract target variable
X = pd.get_dummies(X)  # one-hot encoding of categorical variables

#print(X)
X

,age,balance,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,261,1,-1,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
1,44,29,151,1,-1,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
2,33,2,76,1,-1,0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,1
3,47,1506,92,1,-1,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
4,33,1,198,1,-1,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,825,977,3,-1,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
45207,71,1729,456,2,-1,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
45208,72,5715,1127,5,184,3,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
45209,57,668,508,4,-1,0,0,1,0,0,...,1,0,1,0,1,0,0,0,0,1


In [20]:

scaler = StandardScaler()  # instantiate a scaler
X = scaler.fit_transform(X)  # scale the features

X

array([[ 1.60696496,  0.25641925,  0.0110161 , ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [ 0.28852927, -0.43789469, -0.41612696, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [-0.74738448, -0.44676247, -0.70736086, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       ...,
       [ 2.92540065,  1.42959305,  3.37379688, ..., -0.20597248,
         5.37784754, -2.11631591],
       [ 1.51279098, -0.22802402,  0.97014641, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [-0.37068857,  0.52836436,  0.39932797, ...,  4.85501757,
        -0.185948  , -2.11631591]])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [31]:
# Training the SVM model
svm = SVC(kernel='rbf', C=1, gamma=0.1)
svm.fit(X_train, y_train)

SVC(C=1, gamma=0.1)

In [37]:
from sklearn.metrics import classification_report

y_pred = svm.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          no       0.88      1.00      0.94      7996
         yes       0.00      0.00      0.00      1047

    accuracy                           0.88      9043
   macro avg       0.44      0.50      0.47      9043
weighted avg       0.78      0.88      0.83      9043



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
accuracy = accuracy_score(y_test, y_pred)

print(f'The accuracy of the SVM model is {accuracy:.2f}')

The accuracy of the SVM model is 0.88
